# LP

In [2]:
import datetime
import numpy as np
import pandas as pd
import joblib
import warnings
import logging
import os
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import collections
import re
import copy
import json
import pulp

import utils

from tqdm import tqdm
from dateutil.relativedelta import relativedelta
from joblib import Parallel, delayed


pd.set_option('display.max_columns', None)
pd.set_option('max_row', 500)
warnings.filterwarnings('ignore')
tqdm.pandas(desc='pandas bar')

In [ ]:
df_condition_score_group_sum = utils.load_pickle('df_condition_score_group.pickle')
print(df_condition_score_group_sum.shape)
df_condition_score_group_sum.head()

In [ ]:
df_condition_score_group_count = utils.load_pickle('df_condition_score_group_count.txt')
print(df_condition_score_group_count.shape)
df_condition_score_group_count.head()

In [ ]:
df_condition_score_group = df_condition_score_group_sum.\
    merge(df_condition_score_group_count, on=['aaa', 'bbb', 'ccc'], how='left')
print(df_condition_score_group.shape)
df_condition_score_group.head()

In [ ]:
func = pulp.LpProblem('coupon', sense=pulp.LpMaximize)
func

In [ ]:
var_choices = pulp.LpVariable.dicts('Choices', (range(df_condition_score_group.shape[0]), range(5)), 
                                    lowBound=0, upBound=1, cat=pulp.LpBinary)
var_choices

In [ ]:
# 约束1：高、中、低活人群补贴金额范围限定，隐式写在决策变量中
# 高活人群：1.0，1.5，1.8
# 中活人群：0.5，1.0，1.5
# 低活人群：0.2，0.5，1.0
# 高活
field_2_index = df_condition_score_group[df_condition_score_group['ccc']==2].index.values
for i in field_2_index:
    for j in [0, 1]:
        var_choices[i][j].upBound = 0
# 中活
field_1_index = df_condition_score_group[df_condition_score_group['ccc']==1].index.values
for i in field_1_index:
    for j in [0, 4]:
        var_choices[i][j].upBound = 0
# 低活
field_0_index = df_condition_score_group[df_condition_score_group['ccc']==0].index.values
for i in field_0_index:
    for j in [3, 4]:
        var_choices[i][j].upBound = 0
        
for i in range(df_condition_score_group.shape[0]):
    print(df_condition_score_group.loc[i, 'ccc'], [(var_choices[i][j].lowBound,var_choices[i][j].upBound) for j in range(5)])

In [ ]:
df_ctr_score = df_condition_score_group[['score_coupon_02', 'score_coupon_05', 'score_coupon_10', 'score_coupon_15', 'score_coupon_18']]
print(df_ctr_score.shape)
df_ctr_score.head()

In [ ]:
rows = df_ctr_score.shape[0]
cols = df_ctr_score.shape[1]
print(rows)
print(cols)

In [ ]:
# 目标函数
func += pulp.lpSum([var_choices[i][j]*df_ctr_score.iloc[i, j] 
                    for i in range(rows) 
                    for j in range(cols)]), 'maximize ctr score'
func

In [ ]:
# 约束2：每组只有一张券被选择
for i in range(rows):
    func += pulp.lpSum([var_choices[i][j] for j in range(cols)]) == 1, ''

In [ ]:
func.constraints

In [ ]:
# 约束3：CAC
df_cost = df_condition_score_group[['count']].copy()
df_cost['cost_0'] = df_cost['count'] * 0.2
df_cost['cost_1'] = df_cost['count'] * 0.5
df_cost['cost_2'] = df_cost['count'] * 1.0
df_cost['cost_3'] = df_cost['count'] * 1.5
df_cost['cost_4'] = df_cost['count'] * 1.8
df_cost.head()

In [ ]:
df_cost = df_cost[['cost_0', 'cost_1', 'cost_2', 'cost_3', 'cost_4']]
print(df_cost.shape)
df_cost.head()

In [ ]:
func += pulp.lpSum([var_choices[i][j]*df_cost.iloc[i, j] 
                    for i in range(rows) 
                    for j in range(cols)]) <= 1.0 * df_condition_score_group['count'].sum(), 'CAC'

In [ ]:
func.constraints['CAC']

In [ ]:
func.writeLP('coupon.lp')

In [ ]:
func.solve()

In [ ]:
pulp.LpStatus[func.status]

In [ ]:
df_condition_score_group['coupon'] = -1
list_coupon = [0.2, 0.5, 1.0, 1.5, 1.8]
for i in range(rows):
    sum_row = 0
    for j in range(cols):
        sum_row += pulp.value(var_choices[i][j])
        if sum_row > 1:
            raise Exception('Wrong!!!')
        if pulp.value(var_choices[i][j]) == 1:
            df_condition_score_group.loc[i, 'coupon'] = list_coupon[j]
df_condition_score_group.head()

In [ ]:
df_condition_score_group['coupon'].value_counts()

In [ ]:
# result limit
print('capacity: {c}'.format(c=(df_condition_score_group['coupon']*df_condition_score_group['count']).sum()))
print('CAC: {cac}'.format(cac=df_condition_score_group['coupon'].agg(func=np.average, axis=0, weights=df_condition_score_group['count'])))

In [ ]:
# 分配结果check
# 低活
df_condition_score_group[(df_condition_score_group['ccc']==0)&
                         (df_condition_score_group['coupon']>1.0)]

In [ ]:
# 中活
df_condition_score_group[(df_condition_score_group['ccc']==1)&
                         ((df_condition_score_group['coupon']<0.5)|
                         (df_condition_score_group['coupon']>1.5))]

In [ ]:
# 高活
df_condition_score_group[(df_condition_score_group['ccc']==2)&
                         (df_condition_score_group['coupon']<1.0)]

In [ ]:
utils.save_pickle(df_condition_score_group, 'df_csg_result.pickle')

In [ ]:
# upload
df_csg_result = utils.load_pickle('df_csg_result.pickle')
print(df_csg_result.shape)
df_csg_result.head()

In [ ]:
df_csg_result = df_csg_result[['aaa', 'bbb', 'ccc', 'coupon']].copy()
print(df_csg_result.shape)
df_csg_result.head()

In [ ]:
df_csg_result['aaa'] = df_csg_result['aaa'].astype(int).astype(str)
df_csg_result['aaa'].replace('-1', '', inplace=True)

df_csg_result.rename(columns={'bbb': 'bbb_'}, inplace=True)
df_csg_result['bbb_'] = df_csg_result['bbb_'] + 1
df_csg_result['bbb_'] = df_csg_result['bbb_'].astype(int).astype(str)

df_csg_result['ccc'] = df_csg_result['ccc'].astype(int)

df_csg_result['coupon'] = (df_csg_result['coupon']*100).astype(int)

df_csg_result.head(30)

In [ ]:
df_csg_result.info()

In [ ]:
df_csg_result['coupon'].value_counts()

In [ ]:
df_cw_group = df_csg_result.groupby(by=['aaa', 'bbb_'])

list_df_cwj = []
for name, group in df_cw_group:
    dict_na_c = group[['ccc', 'coupon']].to_dict(orient='list')
    dict_field = {x[0]:x[1] for x in zip(dict_na_c['ccc'], dict_na_c['coupon'])}
    dict_field_lmh = {}
    for k, v in dict_field.items():
        if k == 0:
            dict_field_lmh['low'] = v
        elif k == 1:
            dict_field_lmh['mid'] = v
        else:
            dict_field_lmh['high'] = v
    df_cwj_each = pd.DataFrame({'aaa': [name[0]], 
                                'bbb_': [name[1]], 
                                'field': [str(dict_field_lmh)]})
    list_df_cwj.append(df_cwj_each)

df_cwj = pd.concat(list_df_cwj, axis=0)
df_cwj.reset_index(drop=True, inplace=True)
print(df_cwj.shape)
df_cwj.head(10)

In [ ]:
df_cwj.info()

In [ ]:
utils.save_pickle(df_cwj, 'df_cwj_upload_cac100.pickle')
df_cwj.to_csv('df_cwj_upload_cac100.txt', sep='\t', encoding='utf-8', index=False, header=False)

In [ ]:
print('uplift compare best limit(1.04): {u}'.format(u=(2996092.25195659-3021637.256207411)/3021637.256207411))
print('uplift compare best(1.799996)  : {u}'.format(u=(2996092.25195659-3930707.753239899)/3930707.753239899))
print('uplift compare 1.8             : {u}'.format(u=(2996092.25195659-3930707.34061152)/3930707.34061152))
print('uplift compare 1.5             : {u}'.format(u=(2996092.25195659-3848120.547460424)/3848120.547460424))
print('uplift compare 1.0             : {u}'.format(u=(2996092.25195659-2992858.743431806)/2992858.743431806))
print('uplift compare 0.5             : {u}'.format(u=(2996092.25195659-2183640.968774476)/2183640.968774476))
print('uplift compare 0.2             : {u}'.format(u=(2996092.25195659-1789182.0306516674)/1789182.0306516674))
print('uplift compare mckp(0.99998)   : {u}'.format(u=(2996092.25195659-2996073.071639568)/2996073.071639568))